In [2]:
import numpy as np
import pandas as pd

In [3]:
data_train = pd.read_csv('../Datasets/Fashion MNIST/fashion-mnist_train.csv')
data_train

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,1,0,0,0,0,0,0,0,0,0,...,73,0,0,0,0,0,0,0,0,0
59997,8,0,0,0,0,0,0,0,0,0,...,160,162,163,135,94,0,0,0,0,0
59998,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
x_train = data_train.drop(['label'], axis=1).to_numpy()
x_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [7]:
y_train = data_train['label'].to_numpy()
y_train

array([2, 9, 6, ..., 8, 8, 7], dtype=int64)

In [4]:
data_train['label'].value_counts()

label
2    6000
9    6000
6    6000
0    6000
3    6000
4    6000
5    6000
8    6000
7    6000
1    6000
Name: count, dtype: int64

In [1]:
import numpy as np

class CNN:
    def __init__(self, input_size, num_classes):
        # Инициализация параметров сети
        self.input_size = input_size
        self.num_classes = num_classes
        # Инициализация фильтров и весов
        self.filters1 = np.random.randn(3, 3, 1, 32)  # Фильтры первого сверточного слоя
        self.filters2 = np.random.randn(3, 3, 32, 64)  # Фильтры второго сверточного слоя
        self.weights_fc = np.random.randn(7*7*64, 100)  # Веса полносвязного слоя
        self.weights_output = np.random.randn(100, num_classes)  # Веса выходного слоя

    def convolve2d(self, input_data, filters):
        # Метод для выполнения операции свертки
        depth, height, width, _ = input_data.shape
        filter_height, filter_width, _, num_filters = filters.shape

        output_height = height - filter_height + 1
        output_width = width - filter_width + 1

        output_data = np.zeros((depth, output_height, output_width, num_filters))

        for i in range(output_height):
            for j in range(output_width):
                for k in range(num_filters):
                    output_data[:, i, j, k] = np.sum(input_data[:, i:i+filter_height, j:j+filter_width, :] * filters[:, :, :, k])

        return output_data

    def max_pooling(self, input_data, pool_size=(2, 2)):
        # Метод для выполнения операции полумаксимального пулинга
        depth, height, width, num_filters = input_data.shape
        pool_height, pool_width = pool_size

        output_height = height // pool_height
        output_width = width // pool_width

        output_data = np.zeros((depth, output_height, output_width, num_filters))

        for i in range(output_height):
            for j in range(output_width):
                output_data[:, i, j, :] = np.max(input_data[:, i*pool_height:(i+1)*pool_height, j*pool_width:(j+1)*pool_width, :], axis=(1, 2))

        return output_data

    def flatten(self, input_data):
        # Метод для распрямления данных
        return input_data.reshape(-1)

    def fully_connected(self, input_data, weights):
        # Метод для выполнения операции полносвязного слоя
        return np.dot(input_data, weights)

    def softmax(self, x):
        # Функция softmax для преобразования в вероятностное распределение
        exp_x = np.exp(x - np.max(x))
        return exp_x / exp_x.sum(axis=0, keepdims=True)

    def train(self, X_train, y_train, learning_rate=0.01, epochs=10):
        # Обучение сети
        for epoch in range(epochs):
            for index, image in enumerate(X_train):
                # Forward pass (прямой проход)
                conv1_output = self.convolve2d(image, self.filters1)
                relu1_output = np.maximum(0, conv1_output)
                pool1_output = self.max_pooling(relu1_output)
                
                conv2_output = self.convolve2d(pool1_output, self.filters2)
                relu2_output = np.maximum(0, conv2_output)
                pool2_output = self.max_pooling(relu2_output)

                flattened_output = self.flatten(pool2_output)
                fc_output = self.fully_connected(flattened_output, self.weights_fc)
                fc_output_relu = np.maximum(0, fc_output)

                output = self.fully_connected(fc_output_relu, self.weights_output)
                predicted_probs = self.softmax(output)

                # Backpropagation (обратное распространение)
                loss_gradient = predicted_probs - y_train[index]
                weights_output_gradient = np.outer(fc_output_relu, loss_gradient)
                fc_output_gradient = np.dot(self.weights_output, loss_gradient)
                fc_output_gradient[fc_output < 0] = 0  # Обратный проход для ReLU
                weights_fc_gradient = np.outer(flattened_output, fc_output_gradient)
                
                # Обновление весов
                self.weights_output -= learning_rate * weights_output_gradient
                self.weights_fc -= learning_rate * weights_fc_gradient.reshape(self.weights_fc.shape)

    def predict(self, X_test):
        # Предсказание на основе обученных весов
        predictions = []
        for image in X_test:
            conv1_output = self.convolve2d(image, self.filters1)
            relu1_output = np.maximum(0, conv1_output)
            pool1_output = self.max_pooling(relu1_output)
            
            conv2_output = self.convolve2d(pool1_output, self.filters2)
            relu2_output = np.maximum(0, conv2_output)
            pool2_output = self.max_pooling(relu2_output)

            flattened_output = self.flatten(pool2_output)
            fc_output = self.fully_connected(flattened_output, self.weights_fc)
            fc_output_relu = np.maximum(0, fc_output)

            output = self.fully_connected(fc_output_relu, self.weights_output)
            predictions.append(output)

        return predictions





NameError: name 'X_train' is not defined

In [13]:
import numpy as np

# Предположим, у вас есть данные в формате (1, 784)
data_flat = np.random.rand(1, 784)

# Преобразование данных в формат (1, 28, 28)
data_reshaped = data_flat.reshape((1, 28, 28))

# Печать размеров до и после преобразования
print("Размеры до преобразования:", data_flat.shape)
print("Размеры после преобразования:", data_reshaped.shape)

Размеры до преобразования: (1, 784)
Размеры после преобразования: (1, 28, 28)


In [15]:
data_reshaped

array([[[0.95079199, 0.63419524, 0.05893826, 0.24577869, 0.08390007,
         0.6321478 , 0.18966093, 0.50862725, 0.28711489, 0.88912394,
         0.78266502, 0.80105849, 0.73945379, 0.66989088, 0.28702201,
         0.28164991, 0.13030875, 0.22469178, 0.93732261, 0.81795374,
         0.00306749, 0.46277253, 0.5578213 , 0.10996344, 0.91715157,
         0.44876643, 0.8594828 , 0.82764354],
        [0.436177  , 0.52612504, 0.17791624, 0.55760162, 0.08173201,
         0.51652771, 0.59878577, 0.58969443, 0.74763204, 0.11525868,
         0.12839477, 0.10516509, 0.39051183, 0.00287968, 0.28873868,
         0.55288493, 0.1647377 , 0.01743091, 0.98334212, 0.14533031,
         0.70818897, 0.982988  , 0.29684946, 0.62852076, 0.15948997,
         0.76839785, 0.40281803, 0.93877749],
        [0.42068325, 0.71305052, 0.15309684, 0.83292431, 0.55645609,
         0.49059853, 0.87396591, 0.47633489, 0.80826604, 0.09380739,
         0.64312796, 0.47349102, 0.57460796, 0.37148263, 0.47055771,
         0.

In [14]:
data_flat

array([[0.95079199, 0.63419524, 0.05893826, 0.24577869, 0.08390007,
        0.6321478 , 0.18966093, 0.50862725, 0.28711489, 0.88912394,
        0.78266502, 0.80105849, 0.73945379, 0.66989088, 0.28702201,
        0.28164991, 0.13030875, 0.22469178, 0.93732261, 0.81795374,
        0.00306749, 0.46277253, 0.5578213 , 0.10996344, 0.91715157,
        0.44876643, 0.8594828 , 0.82764354, 0.436177  , 0.52612504,
        0.17791624, 0.55760162, 0.08173201, 0.51652771, 0.59878577,
        0.58969443, 0.74763204, 0.11525868, 0.12839477, 0.10516509,
        0.39051183, 0.00287968, 0.28873868, 0.55288493, 0.1647377 ,
        0.01743091, 0.98334212, 0.14533031, 0.70818897, 0.982988  ,
        0.29684946, 0.62852076, 0.15948997, 0.76839785, 0.40281803,
        0.93877749, 0.42068325, 0.71305052, 0.15309684, 0.83292431,
        0.55645609, 0.49059853, 0.87396591, 0.47633489, 0.80826604,
        0.09380739, 0.64312796, 0.47349102, 0.57460796, 0.37148263,
        0.47055771, 0.71061395, 0.14766724, 0.94